In [12]:
import xml.etree.ElementTree as ET
import pandas as pd
import zipfile
from lxml import etree
from xml.etree.ElementTree import XML
docxFileName = "quick_test.docx"
docxZip = zipfile.ZipFile(docxFileName)
documentXML = docxZip.read('word/document.xml')
et = etree.XML(documentXML)
ns = {'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main'}

In [20]:
mainstation_xpath = './w:r[1][not(w:rPr/w:sz[@w:val="16"])]/w:t[1]'
lowerstation_xpath = './w:r[1][not(w:rPr/w:sz[@w:val="18"])]/w:t[1]'
first_token_para_xpath = './w:r[1]/w:t[1]'

stations = pd.DataFrame(columns=['station','type','description'])

def station_type (para):
    #if para.xpath(first_token_para_xpath, namespaces=ns)[0].text.islower() == 'True':
        #return 'na'
    if para.xpath(mainstation_xpath, namespaces=ns):
        return 'main'        
    elif para.xpath(lowerstation_xpath, namespaces=ns):
        return 'lower'
    else:
        return 'na'


def is_upper(para):
    if para.xpath(first_token_para_xpath, namespaces=ns):
        if para.xpath(first_token_para_xpath, namespaces=ns)[0].text.isupper():
            return 'station'
        else: 
            return 'new_line'


for i, para in enumerate(et.xpath('//w:p', namespaces=ns)):
    text = para.xpath('./w:r/w:t', namespaces=ns)
    description = " ".join([t.text for t in text]) 
    if is_upper(para) == 'station':
        if station_type(para) =='main':
            mainstation = para.xpath(mainstation_xpath, namespaces=ns)            
            stations.at[i,'station'] = mainstation[0].text
            stations.at[i,'type'] = 'main'
            print("MAIN:", mainstation[0].text)
        elif station_type(para) =='lower':
            lowerstation = para.xpath(lowerstation_xpath, namespaces=ns)
            stations.at[i,'station'] = lowerstation[0].text
            stations.at[i,'type'] = 'secondary'
            print("SUB:", lowerstation[0].text)
    else:
        stations.at[i,'station'] = '-'
        stations.at[i,'type'] = '-'
    stations.at[i,'description'] = description
    print("DESCRIPTION:", description)
    print()


#stations.head(5)
stations.to_csv('output.csv')


##notes: 
#"ABERDARE JUNCTION  see ABERCYNON." ("JUNCTION" in document.xml isn't marked-up correctly)
#"Bridge over line near Schoolhill ticket platform (ibid 8 June 1895). Separate structure or station used as collection point for..." ("Bridge" in document.xml isn't marked-up correctly)
    
    


MAIN: ABBEY
DESCRIPTION: ABBEY

SUB: A TOWN
DESCRIPTION: A TOWN   [NB] op 3 September 1856** as A; TOWN added 1889 tt  (Cl) ; clo 7 September 1964  (RM October).  Initially ABBEYHOLME  (Brad).

SUB: A JUNCTION
DESCRIPTION: A JUNCTION   [Cal] op 31 August 1870  (co ½ T 26 September) . Clo 20 May 1921 (see 192 1  April/ May**).

SUB: A JUNCTION
DESCRIPTION: A JUNCTION   [NB] op 8 August 1870  (D&C 14) ; clo 1 September 1921 (Thursday)  (RCG)  – stops still shown until then in  Brad  ( all days August 1921)  but would seem to have been pointless after April/May 1921** .  L eft in case Cal line reopened? Did trains actually call?

DESCRIPTION: Brad  and co tts seen all have Abbey Junction for the station concerned. OS maps show as effectively one station.

MAIN: ABBEY & WEST DEREHAM 
DESCRIPTION: ABBEY & WEST DEREHAM  [GE] op 1 August 1882  (Thetford & Watton Times 5 th )  as  A; became A for W  D  1  January 1886  (Cl; RCG ref Jan) , A & W D 1 July 1923  (hbl 12 th ) ; clo 22 September 19

In [30]:
import re
mainstation_xpath = './w:r[1][not(w:rPr/w:sz[@w:val="16"])]/w:t[1]'
lowerstation_xpath = './w:r[1][not(w:rPr/w:sz[@w:val="18"])]/w:t[1]'
first_token_para_xpath = './w:r[1]/w:t[1]'

stations = pd.DataFrame(columns=['station','type','description'])

def station_type (para):
    #if para.xpath(first_token_para_xpath, namespaces=ns)[0].text.islower() == 'True':
        #return 'na'
    if para.xpath(mainstation_xpath, namespaces=ns):
        return 'main'        
    elif para.xpath(lowerstation_xpath, namespaces=ns):
        return 'lower'
    else:
        return 'na'


def is_upper(para):
    if para.xpath(first_token_para_xpath, namespaces=ns):
        if para.xpath(first_token_para_xpath, namespaces=ns)[0].text.isupper():
            return 'station'
        else: 
            return 'new_line'

def rail_company(text):
    rail_company = re.findall(r'\[[a-zA-Z]+\]', text)
    return rail_company


for i, para in enumerate(et.xpath('//w:p', namespaces=ns)):
    text = para.xpath('./w:r/w:t', namespaces=ns)
    description = " ".join([t.text for t in text]) 
    if is_upper(para) == 'station':
        if station_type(para) =='main':
            mainstation = para.xpath(mainstation_xpath, namespaces=ns)            
            stations.at[i,'station'] = mainstation[0].text
            stations.at[i,'type'] = 'main'
            print("MAIN:", mainstation[0].text)
        elif station_type(para) =='lower':
            lowerstation = para.xpath(lowerstation_xpath, namespaces=ns)
            stations.at[i,'station'] = lowerstation[0].text
            stations.at[i,'type'] = 'secondary'
            print("SUB:", lowerstation[0].text)
    else:
        stations.at[i,'station'] = '-'
        stations.at[i,'type'] = '-'
    stations.at[i,'rail_company'] = rail_company(description)
    stations.at[i,'description'] = description
    print("DESCRIPTION:", description)
    print()


#stations.head(5)
stations.to_csv('output.csv')


##notes: 
#"ABERDARE JUNCTION  see ABERCYNON." ("JUNCTION" in document.xml isn't marked-up correctly)
#"Bridge over line near Schoolhill ticket platform (ibid 8 June 1895). Separate structure or station used as collection point for..." ("Bridge" in document.xml isn't marked-up correctly)
    
    


MAIN: ABBEY


ValueError: cannot set using a multi-index selection indexer with a different length than the value